In [1]:
import os
import random # Analysis
import datetime as dt # Analysis
import numpy as np # Analysis
import pandas as pd # Analysis
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm

In [23]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings

In [2]:
# datafarme 보여주는 범위 설정

pd.options.display.max_columns=1000
pd.options.display.max_rows=200
pd.options.display.float_format = '{:.5f}'.format

In [31]:
load_path = './data/' # 데이터 불러오는 경로
save_path = './preprocessing/' # 전처리 이후 데이터 저장하는 경로
result_path = './submission/' # inference 결과 저장하는 경로

In [4]:
# train_err = pd.read_csv(save_path + 'train_err_data_rev_220607.csv')
# train_qual = pd.read_csv(save_path + 'train_quality_data_rev_220607.csv')
# train_prob = pd.read_csv(save_path + 'train_problem_data_rev_220607.csv')
# test_err = pd.read_csv(save_path + 'test_err_data_rev_220607.csv')
# test_qual = pd.read_csv(save_path + 'test_quality_data_rev_220607.csv')
labeled_train_err = pd.read_csv(save_path + 'labeled_train_err_data.csv')

In [5]:
labeled_train_err

,user_id,time,model_nm,fwver,errtype,errcode,target
0,10000,2020-11-01 02:56:16,model_3,05.15.2138,15,1,0
1,10000,2020-11-01 03:03:09,model_3,05.15.2138,12,1,0
2,10000,2020-11-01 03:03:09,model_3,05.15.2138,11,1,0
3,10000,2020-11-01 05:05:14,model_3,05.15.2138,16,1,0
4,10000,2020-11-01 05:05:15,model_3,05.15.2138,4,0,0
...,...,...,...,...,...,...,...
16554658,24999,2020-11-30 16:30:51,model_3,05.15.2138,15,1,0
16554659,24999,2020-11-30 17:26:25,model_3,05.15.2138,16,1,0
16554660,24999,2020-11-30 17:26:25,model_3,05.15.2138,4,0,0
16554661,24999,2020-11-30 17:26:31,model_3,05.15.2138,4,0,0


In [8]:
# 데이터 설명을 확인하면
# ueser_id가 10000부터 24999까지 총 15000개가 연속적으로 존재.
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

In [11]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = labeled_train_err[['user_id','errtype']].values
error = np.zeros((len(labeled_train_err),42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:41<00:00, 396344.78it/s]


(16554663, 42)

In [19]:
problem = np.array(labeled_train_err.target)

In [20]:
problem

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [21]:
train_x = error
train_y = problem
del error, problem
print(train_x.shape)
print(train_y.shape)

(16554663, 42)
(16554663,)


In [24]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 3586431, number of negative: 9657299
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.541959 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 631
[LightGBM] [Info] Number of data points in the train set: 13243730, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.270802 -> initscore=-0.990556
[LightGBM] [Info] Start training from score -0.990556
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[5]	valid_0's auc: 0.500182	valid_0's pr_auc: 0.256859


D:\Anaconda\envs\yd\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[LightGBM] [Info] Number of positive: 3585523, number of negative: 9658207
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.369105 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 621
[LightGBM] [Info] Number of data points in the train set: 13243730, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.270734 -> initscore=-0.990904
[LightGBM] [Info] Start training from score -0.990904
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[5]	valid_0's auc: 0.5002	valid_0's pr_auc: 0.230825


D:\Anaconda\envs\yd\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[LightGBM] [Info] Number of positive: 3586348, number of negative: 9657382
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 2.146142 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 634
[LightGBM] [Info] Number of data points in the train set: 13243730, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.270796 -> initscore=-0.990588
[LightGBM] [Info] Start training from score -0.990588
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.50021	valid_0's pr_auc: 0.208394


D:\Anaconda\envs\yd\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[LightGBM] [Info] Number of positive: 3586766, number of negative: 9656965
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.555992 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 621
[LightGBM] [Info] Number of data points in the train set: 13243731, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.270827 -> initscore=-0.990428
[LightGBM] [Info] Start training from score -0.990428
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.500177	valid_0's pr_auc: 0.237755


D:\Anaconda\envs\yd\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[LightGBM] [Info] Number of positive: 3586724, number of negative: 9657007
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.537200 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 610
[LightGBM] [Info] Number of data points in the train set: 13243731, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.270824 -> initscore=-0.990445
[LightGBM] [Info] Start training from score -0.990445
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[6]	valid_0's auc: 0.500205	valid_0's pr_auc: 0.2648


D:\Anaconda\envs\yd\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
print(np.mean(auc_scores))

0.5001948109753992


In [27]:
test_err = pd.read_csv(save_path + 'test_err_data_rev_220607.csv')
display(test_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1


In [42]:
test_err

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1
...,...,...,...,...,...,...
16532643,44998,2020-11-30 21:00:50,model_1,04.16.3553,40,0
16532644,44998,2020-11-30 21:18:31,model_1,04.16.3553,31,1
16532645,44998,2020-11-30 21:18:32,model_1,04.16.3553,15,1
16532646,44998,2020-11-30 21:22:59,model_1,04.16.3553,16,1


In [30]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((len(test_err),42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - len(test_err), err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:53<00:00, 307559.84it/s]

(16532648, 42)


In [52]:
temp1 = pd.DataFrame({"user_id" : id_error[:, 0], "errtype" : id_error[:, 1]})
temp1

,user_id,errtype
0,30000,31
1,30000,33
2,30000,15
3,30000,22
4,30000,11
...,...,...
16532643,44998,40
16532644,44998,31
16532645,44998,15
16532646,44998,16


In [53]:
temp1.user_id.unique()

array([30000, 30001, 30002, ..., 44996, 44997, 44998], dtype=int64)

In [70]:
temp1.user_id.nunique()

14998

In [79]:
user_errors = [0] * 14999
for u_id in tqdm(temp1.user_id.unique()):
    user_errors[u_id - 30000] = len(temp1[temp1.user_id == u_id])

100%|███████████████████████████████████████████████████████████████████████████| 14998/14998 [02:23<00:00, 104.78it/s]


In [80]:
print(len(user_errors))
user_errors

14999


[2750,
 284,
 941,
 371,
 881,
 3847,
 160,
 146,
 1092,
 394,
 568,
 745,
 1879,
 202,
 673,
 668,
 1227,
 5636,
 1706,
 1002,
 732,
 1144,
 567,
 221,
 856,
 1360,
 715,
 166,
 2904,
 625,
 530,
 878,
 437,
 642,
 964,
 144,
 640,
 746,
 564,
 98,
 449,
 165,
 171,
 413,
 443,
 15774,
 824,
 403,
 928,
 355,
 802,
 899,
 504,
 355,
 39,
 449,
 438,
 2168,
 1950,
 842,
 1959,
 794,
 1462,
 23,
 544,
 495,
 496,
 874,
 1146,
 682,
 3915,
 1175,
 1348,
 525,
 367,
 568,
 1107,
 473,
 724,
 137,
 1390,
 50,
 171,
 550,
 530,
 675,
 879,
 313,
 710,
 704,
 1246,
 342,
 5,
 549,
 808,
 174,
 385,
 1235,
 1110,
 1648,
 874,
 826,
 1276,
 396,
 404,
 773,
 672,
 1625,
 350,
 300,
 505,
 1001,
 327,
 2539,
 262,
 35,
 836,
 363,
 610,
 525,
 1270,
 581,
 2319,
 233,
 1192,
 408,
 2289,
 66,
 1780,
 1084,
 2076,
 281,
 702,
 693,
 808,
 955,
 882,
 1066,
 1184,
 186,
 3896,
 646,
 459,
 972,
 590,
 92,
 2420,
 3741,
 1536,
 588,
 84,
 2533,
 43,
 1275,
 602,
 653,
 533,
 177,
 1227,
 229,
 681

In [32]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [38]:
print(len(np.unique(pred_ensemble)))
np.unique(pred_ensemble)

10261


array([0.215799  , 0.21639332, 0.21665144, ..., 0.31317418, 0.31710657,
       0.32423915])

In [66]:
pred_ensemble

array([[0.27082238],
       [0.27082238],
       [0.27082238],
       ...,
       [0.27082238],
       [0.27082238],
       [0.27082238]])

In [60]:
pred_ensemble[0:2750].mean()

0.2708223784132864

In [81]:
pred = []
idx = 0
for n in tqdm(user_errors):
    pred.append(pred_ensemble[idx:idx + n].mean())
    idx += n

 83%|███████████████████████████████████████████████████████████▉            | 12490/14999 [00:00<00:00, 124098.30it/s]C:\Users\JunNyeong Jeong\AppData\Local\Temp\ipykernel_17520\1604744341.py:4: RuntimeWarning: Mean of empty slice.
  pred.append(pred_ensemble[idx:idx + n].mean())
D:\Anaconda\envs\yd\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|█████████████████████████████████████████████████████████████████████████| 14999/14999 [00:00<00:00, 91368.17it/s]


In [85]:
np.unique(pred)

array([0.24185385, 0.24214244, 0.24235285, 0.24238703, 0.24244771,
       0.24247622, 0.24266062, 0.24276215, 0.2427683 , 0.24287557,
       0.2429489 , 0.24295663, 0.24308984, 0.24315277, 0.24316521,
       0.24323132, 0.24337694, 0.24357662, 0.24362236, 0.2650278 ,
       0.26971183, 0.27082238, 0.27082238, 0.27082238, 0.27082238,
       0.27082238, 0.27082238, 0.27082238, 0.27082238,        nan])

In [64]:
sample_submssion = pd.read_csv(load_path + 'sample_submission.csv')

In [86]:
sample_submssion['problem'] = np.array(pred).reshape(-1)

In [97]:
sample_submssion = sample_submssion.fillna(0)

In [98]:
sample_submssion.isnull().sum()

user_id    0
problem    0
dtype: int64

In [100]:
sample_submssion.describe()

,user_id,problem
count,14999.00000,14999.00000
mean,37499.00000,0.27077
std,4329.98268,0.00243
min,30000.00000,0.00000
25%,33749.50000,0.27082
50%,37499.00000,0.27082
75%,41248.50000,0.27082
max,44998.00000,0.27082


In [101]:
sample_submssion.to_csv(result_path + 'submission_220609-1.csv', index=False)